## Implementation of MimiFAIR162-MimiFUND

This notebook implements a coupling of the MimiFAIR model and the MimiFUND model. At a high-level, the goal here is to use the MimiFAIR model as the climate module, and the MimiFUND model as the socioeconomic and damages modules, although in this case we still allow MimiFUND's climate module to take care of the modeling of non-CO₂ gases like methane.  

This is meant to be an instructional example for the use-case of Mimi where a user wishes to couple together existing models.

Resources for further inquiries include:

- The Mimi framework [website](https://www.mimiframework.org)
- The Mimi [forum](https://forum.mimiframework.org) for technical and Mimi-related questions
- The model-specific repositories, [MimiFUND](https://github.com/fund-model/MimiFUND.jl) and [MimiFAIR](https://github.com/anthofflab/mimi-fair.jl/) for model-specific inquiries

If you wish to follow alone, feel free to download or clone the Github repository, or create a new project on your own machine. If you are new to Mimi, you will probably want to start with our basic Mimi installation process as described in our documentation [here](https://www.mimiframework.org/Mimi.jl/stable/tutorials/tutorial_1/#Tutorial-1:-Install-Mimi-1).

### Set up the Environment

The first step is to set up the environment with the required packages and versions for this exercise as indicated in the `Manifest.toml` and `Project.toml` files. As you can see from the call to `Pkg.status()`, this work requires a few packages including the `master` branch of `Mimi`, the default (latest tagged) versions of `MimiFAIR` and `MimiFUND` model implementations, and some supporting packages `CSVFiles`, `DataFrames`, and `Vegalite`.

In [1]:
#using Pkg
#Pkg.activate(@__DIR__)
#Pkg.status()

Note that if you are following along on your own machine without downloading this repository, you can recreate this environment by adding each of the packages listed above with `Pkg.add()`, or using the `Pkg` REPL entered with `]`.

In [2]:
# using Pkg
# Pkg.add("CSVFiles")
# Pkg.add("DataFrames")
# Pkg.add("Mimi")
#Pkg.add("MimiFAIR")
#Pkg.add("MimiFAIRv1_6_2")
# Pkg.add("MimiFUND")
# Pkg.add("VegaLite")

# Pkg.status()

Next we load the packages whose APIs we will be directly using as well as set some constants to be used in the script.

In [3]:
using MimiFAIRv1_6_2
using Mimi

### Load Helpers and Set Constants

We will now load some helper functions and set some constants, most of which have to do with the difference in time dimensions between FAIR and FUND, which will drive much of our coupling work.


The `helper.jl` file defines the `update_MimiFAIR_params!` function with the following signature.  This is copied directly from the source code of `MimiFAIR.jl` and takes care of the calls to `update_param!`, `add_shared_param!`, and `connect_param!` that build up the stand alone MimiFAIR model.


```
function update_MimiFAIR_params!(m; rcp_scenario::String="RCP85", start_year::Int=1765, end_year::Int=2500, F2x::Float64=3.71, TCR::Float64=1.6, ECS::Float64=2.75, d::Array{Float64,1}=[239.0, 4.1])
```

In the future we can likely shorten this function by adding `default` values to the scalar parameters of FAIR.

In [4]:
include("src/helper.jl");
include("E:/10Julia/02FUND-Province-julia/src/FUND-province-julia.jl")

Main.FUNDprovince

In [5]:
# set some constants
ar6_scenario = "ssp585"

FAIR_first = 2015
FAIR_last = 2100
FAIR_len = length(FAIR_first:FAIR_last)

FUND_first = 2015
FUND_last = 2100
FUND_len = length(FUND_first:FUND_last)

86

### Load Independent (Default) FUND and FAIR Models

We begin with the default FUND model as implemented in `MimiFUND.jl`.

In [6]:
using DelimitedFiles
regionsdata = readdlm("E:/10Julia/02FUND-Province-julia/data/regions.csv", ',',String)
regions =regionsdata[2:end, 1]
default_datadir = joinpath("E:/10Julia/02FUND-Province-julia", "data")
default_params = nothing
m = FUNDprovince.get_model(syear=2015, nsteps = 85, regions = regions, datadir = default_datadir, params = default_params)

:emissions:impactaggregation:scenariouncertainty:climatedynamics:climateco2cycle:emissions:scenariouncertainty:climateregional:climateco2cycle:emissions:climatedynamics:population:climateco2cycle:climateco2cycle:climatedynamics:climateco2cycle:scenariouncertainty:socioeconomic:scenariouncertainty

Mimi.Model
  Module: Mimi
  Components:
    ComponentId(Main.FUNDprovince.scenariouncertainty)
    ComponentId(Main.FUNDprovince.population)
    ComponentId(Main.FUNDprovince.socioeconomic)
    ComponentId(Main.FUNDprovince.emissions)
    ComponentId(Main.FUNDprovince.climateco2cycle)
    ComponentId(Main.FUNDprovince.climateforcing)
    ComponentId(Main.FUNDprovince.climatedynamics)
    ComponentId(Main.FUNDprovince.climateregional)
    ComponentId(Main.FUNDprovince.impactaggregation)
  Built: false


### Add and Update Dimensions

The first step in building out our coupled model, starting with our default FUND model, is to handle dimensions. Currently the model `m` has two dimensions, `time` and `regions`. 

In [7]:
m.md.dim_dict # display the dimensions of the default FUND model

OrderedCollections.OrderedDict{Symbol, Union{Nothing, Mimi.Dimension}} with 3 entries:
  :time       => [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024  ……
  :regions    => ["4", "12", "20", "24", "28", "31", "44", "48", "51", "52"  … …
  Symbol("5") => [1, 2, 3, 4, 5]

We now add two new dimensions to our model `m` that are used by FAIR.

In [8]:
# add new dimensions relevant to FAIR (1) minor greenhouse gases and 
# (2) ozone-depleting substances
other_ghg_names = ["CF4", "C2F6", "C6F14", "HFC23", "HFC32", "HFC43_10", "HFC125", "HFC134a", "HFC143a", "HFC227ea", "HFC245fa", "SF6"]
ods_names       = ["CFC_11", "CFC_12", "CFC_113", "CFC_114", "CFC_115", "CARB_TET", "MCF", "HCFC_22", "HCFC_141B", "HCFC_142B", "HALON1211", "HALON1202", "HALON1301", "HALON2402", "CH3BR", "CH3CL"]

set_dimension!(m, :other_ghg, other_ghg_names) # FAIR dims - Set index for Kyoto and ozone ozone-depleting gases.
set_dimension!(m, :ozone_depleting_substances, ods_names) # FAIR dims - Set index for Kyoto and ozone ozone-depleting gases.

m.md.dim_dict  # display the (augmented) dimensions of the default FUND model's Model Definition

OrderedCollections.OrderedDict{Symbol, Union{Nothing, Mimi.Dimension}} with 5 entries:
  :time                       => [2015, 2016, 2017, 2018, 2019, 2020, 2021, 202…
  :regions                    => ["4", "12", "20", "24", "28", "31", "44", "48"…
  Symbol("5")                 => [1, 2, 3, 4, 5]
  :other_ghg                  => ["CF4", "C2F6", "C6F14", "HFC23", "HFC32", "HF…
  :ozone_depleting_substances => ["CFC_11", "CFC_12", "CFC_113", "CFC_114", "CF…

The final dimensions-related step is to expand the time dimension of the model.  At this stage, our model runs on FUND's default time dimension 1950 to 3000 (`FUND_first` to `FUND_last`).  FAIR, however, runs from 1765 to 2500 (`FAIR_first` to `FAIR_last`) so we are going to set our model to run from 1765 to 2500.

FUND will not start running until 1950, and will not produce an `emissions` value until 1951, so up until 1951 FAIR will use `backup` values pulled from default FUND as we will describe in detail below.

**For further details on the effects and restrictions around updating the `time` dimension of a model see the documentation [here](https://www.mimiframework.org/Mimi.jl/dev/howto/howto_5/).**

In [9]:
set_dimension!(m, :time, collect(FAIR_first:FUND_last))

m.md.dim_dict # display the (udpated) dimensions of the model

OrderedCollections.OrderedDict{Symbol, Union{Nothing, Mimi.Dimension}} with 5 entries:
  :time                       => [2015, 2016, 2017, 2018, 2019, 2020, 2021, 202…
  :regions                    => ["4", "12", "20", "24", "28", "31", "44", "48"…
  Symbol("5")                 => [1, 2, 3, 4, 5]
  :other_ghg                  => ["CF4", "C2F6", "C6F14", "HFC23", "HFC32", "HF…
  :ozone_depleting_substances => ["CFC_11", "CFC_12", "CFC_113", "CFC_114", "CF…

### Augment FUND with FAIR Components

We now add the FAIR components to our model.  We place them in a contiguous block starting after the FUND `emissions` component, which will feed results into the FAIR components before FAIR feeds back into FUND from its `temperature` component.

Also note that since we have updated the time dimension to run from `FAIR_first` to `FAIR_last`, or 1765 to 2500, each of these new components will be set to run from 1765 to 2500, the full model time period, by default.

In [10]:
add_comp!(m, MimiFAIRv1_6_2.ch4_cycle; after = :emissions);
add_comp!(m, MimiFAIRv1_6_2.n2o_cycle; after = :ch4_cycle);
add_comp!(m, MimiFAIRv1_6_2.co2_cycle; after = :n2o_cycle);
add_comp!(m, MimiFAIRv1_6_2.other_ghg_cycles; after = :co2_cycle);
add_comp!(m, MimiFAIRv1_6_2.o3_depleting_substance_cycles; after = :other_ghg_cycles);
add_comp!(m, MimiFAIRv1_6_2.co2_forcing; after = :o3_depleting_substance_cycles);
add_comp!(m, MimiFAIRv1_6_2.ch4_forcing; after = :co2_forcing);
add_comp!(m, MimiFAIRv1_6_2.n2o_forcing; after = :ch4_forcing);
add_comp!(m, MimiFAIRv1_6_2.o3_forcing; after = :n2o_forcing);
add_comp!(m, MimiFAIRv1_6_2.aerosol_direct_forcing; after = :o3_forcing);
add_comp!(m, MimiFAIRv1_6_2.aerosol_indirect_forcing; after = :aerosol_direct_forcing);
add_comp!(m, MimiFAIRv1_6_2.other_ghg_forcing; after = :aerosol_indirect_forcing);
add_comp!(m, MimiFAIRv1_6_2.o3_depleting_substance_forcing; after = :other_ghg_forcing);
add_comp!(m, MimiFAIRv1_6_2.contrails_forcing; after = :o3_depleting_substance_forcing);
add_comp!(m, MimiFAIRv1_6_2.bc_snow_forcing; after = :contrails_forcing);
add_comp!(m, MimiFAIRv1_6_2.landuse_forcing; after = :bc_snow_forcing);
add_comp!(m, MimiFAIRv1_6_2.total_forcing; after = :landuse_forcing);
add_comp!(m, MimiFAIRv1_6_2.temperature; after = :total_forcing);

Next we run our helper function to update all the FAIR component parameters and make their internal connections, such that we have added  a fully functional and runnable FAIR model into our FUND model.


In [ ]:
update_MimiFAIR162_params!(m; start_year = FAIR_first, end_year = FAIR_last, ar6_scenario = ar6_scenario)

At this point the model `m` is runnable (and in fact none of the steps in this notebook thus far have made an invalid, unrunnable model), but currently `m` will run FUND and FAIR in their default modes and completely disconnected.  The next section describes the steps to couple them together.

### Couple FUND and FAIR

#### FUND Emissions --> Convert Mtons to Gtons --> FAIR CO₂ Cycle

We want to connect the FUND `emissions` component's `mco2` variable the FAIR `co2_cycle` component's `E_CO₂` parameter.  In the stand-alone version of FAIR, the `E_CO₂` is exogenously forced by the summation of all `CO₂` sources, as shown in it's source code (and the `helper.jl` file):

```
rcp_emissions, volcano_forcing, solar_forcing, gas_data, gas_fractions, conversions = MimiFAIR.load_fair_data(start_year, end_year, rcp_scenario)

update_param!(m, :co2_cycle, :E_CO₂, rcp_emissions.FossilCO2 .+ rcp_emissions.OtherCO2)
```

First we need to add a `multiplier` component between FUND's `emissions` component and the FAIR components because FUND outputs Mtons of C but FAIR expects Gtons of C. **Note that these models expect C instead of CO₂, which is endogenous to the model so we must align with those**.

Mimi has a few built in components for this type of situation, including `Mimi.multiplier`, as decribed [here](https://www.mimiframework.org/Mimi.jl/stable/tutorials/tutorial_3/#Component-and-Structural-Modifications:-The-API-1).  

This small component just needs a connected `input` parameter and a `multiply` parameter to produce an `output` variable. In this case, we connect the `multiplier` component's `input` parameter to the `emissions` component's `mco2` variable and set `multiply` parameter to our unit conversion value of 1/1000. We also set the `multiplier` component to run only while FUND runs by setting `first = FUND_first` and `last = FUND_last`.

In [ ]:
add_comp!(m, Mimi.multiplier; after = :emissions, first = FUND_first, last = FUND_last);
set_param!(m, :multiplier, :multiply, fill(1/1000, FAIR_len)) # convert Mtons C coming out of FUND to Gtons C going into FAIR
connect_param!(m, :multiplier, :input, :emissions, :mco2)

Now that we have taken care of unit conversion, we can connect the FAIR `co2_cycle` component's `E_CO₂` parameter to the `multiplier` component's `output` variable.  A few notes on the arguments having to do with backup data are:

- Since FAIR runs from 1765 to 1950, but FUND (and the multiplier) only runs from 1950 to 2500, we need to include backup data `FAIR_CO₂_backup` for FAIR to use until FUND can provide a value.  We use the same data that previously exogenously forced FAIR.


- We also need to include the `backup_offset` optional argument for the `connect_param!` function, because FUND does not actually produce a value for `mco2` until it's *second* timestep, or in this case 1951, and thus we set `backup_offset = 1`.

In [ ]:
# destination: FAIR component :co2_cycle parameter :E_CO₂
# previous source: exogenous parameter
# new source: FUND :emissions variable :mco2 (which first runs through the multiplier component)

# Subset AR6 emissions to proper years.
ar6_emissions_raw = DataFrame(load(joinpath(@__DIR__, "data", "model_data", "AR6_emissions_"*ar6_scenario*"_1750_2300.csv")))
emission_indices = indexin(collect(FAIR_first:FAIR_last), ar6_emissions_raw.Year)
ar6_emissions = ar6_emissions_raw[emission_indices, :]

FAIR_CO₂_backup = (ar6_emissions.FossilCO2 .+ ar6_emissions.OtherCO2)
connect_param!(m, :co2_cycle, :E_co2, :multiplier, :output, FAIR_CO₂_backup, backup_offset = 1)

A note here is that FAIR v1.6.2 splits land use CO2 emissions and fossil CO2 emissions, and feeds their sum into the `co2_cycle`, but just the land use emissions into the `landuse_forcing` component in MimiFAIRv1_6_2. Given FUND does not explicilty provide the land use emissions (AFOLU), we do not modify the latter connection, and thus FAIR will run with its default land use emissions for the `landuse_forcing` component.

#### FAIR Temperature --> FUND CO₂ Cycle, Biodiversity, Ocean and Regional Climate Components

Now we want to connect FAIR's output temperature variable `T` from component `temperature` back into FUND, both to the `temp` parameter of the `climateco2cycle`, `biodiversity`, and `ocean` components and into the `inputtemp` parameter of the `climateregional` component.

In [ ]:
# destination: FUND components :climateco2cycle, :biodiversity, :ocean --> parameter :temp
# previous source: FUND component :climatedynamics --> variable :temp
# new source: FAIR :temperature --> variable :T

connect_param!(m, :climateco2cycle, :temp, :temperature, :T)
#connect_param!(m, :biodiversity, :temp, :temperature, :T)
#connect_param!(m, :ocean, :temp, :temperature, :T)

In [ ]:
# destination: FUND component :climateregional, parameter :inputtemp
# previous source: FUND component :climatedynamics --> variable :temp
# new source: FAIR component :temperature --> variable :T

connect_param!(m, :climateregional, :inputtemp, :temperature, :T)

### Run and Visualize

Now we can run our coupled model and visualize the results with the `explore`, `plot`, and indexing functions. We will also grab the default FUND and FAIR models for comparisons.

In [ ]:
run(m) # coupled model

using MimiFUND
mfund = MimiFUND.get_model()
run(mfund)

mfair = MimiFAIRv1_6_2.get_model(;start_year = FAIR_first, end_year = FAIR_last, ar6_scenario = ar6_scenario)
run(mfair)

In [ ]:
 #explore(m) # fails in VSCode-run notebook, uncomment if running these commands locally or in a browser!

In [ ]:
scc = FUNDprovince.compute_scco2(m, year = 2020,last_year=2100)
##??? debug

#### Compare Some Time Series

One interesting step might be to compare some time series between the different models.  We could use the `Mimi.plot` function (ie. `Mimi.plot(m, :temperature, :T`) but here we will use some custom written plot code with the `VegaLite` package.

In [ ]:
using VegaLite
using DataFrames

##### Temperature

Here we compare the delta temperature trajectory between FAIR, FUND, and FUNDFAIR. Noting that from 1765 to 1950 our results in FUNDFAIR will match FAIR, and then in 1950 FUND will kick in as our driver of CO2 emissions calculations while FAIR will continue to drive temperature calculations for which it recieves CO2 emissions inputs from FUND.

In [ ]:
fairvals = mfair[:temperature, :T]
fundfairvals =  m[:temperature, :T]
fundvals = vcat(
    fill(missing, length(FAIR_first:FUND_first)-1),
    mfund[:climateco2cycle, :temp][1:length(FUND_first:FAIR_last),:]
)
fundvals = fundvals[:,1]; # make a vector

In [ ]:
df = DataFrame(
    :Year => Mimi.time_labels(m),
    :FUND => fundvals,
    :FAIR => fairvals,
    :FUNDFAIR => fundfairvals
)

stack(df, [:FUND, :FAIR, :FUNDFAIR]) |> 

@vlplot(
    :line, 
    x = :Year,
    y = :value,
    color = :variable
)
    

##### Loss

Similarly, here we compare what the loss looks like when we use the FUND climate module for CO2 as compared to using the FAIR climate module for CO2.

In [ ]:
fundfairvals = sum(m[:impactaggregation, :loss], dims = 2)[:,1];

fundvals = vcat(
    fill(missing, length(FAIR_first:FUND_first)-1),
    sum(mfund[:impactaggregation, :loss][1:length(FUND_first:FAIR_last),:], dims = 2)
)
fundvals = fundvals[:,1]; # make a vector
     

In [ ]:
df = DataFrame(
    :Year => Mimi.time_labels(m),
    :FUND => fundvals, 
    :FUNDFAIR => fundfairvals)

stack(df, [:FUND, :FUNDFAIR]) |> 

@vlplot(
    :line, 
    x = :Year,
    y = :value,
    color = :variable
)

#### Quality Checking

We want to make sure that values are being passed between FUND and FAIR in the right units etc. so let's take a closer look at emissions.

##### Annual Carbon Dioxide Emissions (GtC yr⁻¹)

In [ ]:
fairvals = mfair[:co2_cycle, :E_co2]
fundfairvals =  m[:co2_cycle, :E_co2]

df = DataFrame(
    :Year => Mimi.time_labels(m),
    :FAIR => fairvals, 
    :FUNDFAIR => fundfairvals)

stack(df, [:FAIR, :FUNDFAIR]) |> 

@vlplot(
    :line, 
    x = :Year,
    y = :value,
    color = :variable
)

In [ ]:
##### Global Carbon Dioxide Emissions (MtC yr⁻¹)

In [ ]:
fundfairvals =  m[:emissions, :cumglobco2]
fundvals = vcat(
    fill(missing, length(FAIR_first:FUND_first)-1),
    sum(mfund[:emissions, :cumglobco2][1:length(FUND_first:FAIR_last),:], dims = 2)
)
fundvals = fundvals[:,1]; # make a vector

df = DataFrame(
    :Year => Mimi.time_labels(m),
    :FUND => fundvals, 
    :FUNDFAIR => fundfairvals)

stack(df, [:FUND, :FUNDFAIR]) |> 

@vlplot(
    :line, 
    x = :Year,
    y = :value,
    color = :variable
)

In [ ]:
# difference between fundfair and fund in cumulative emissions since the graph above masks it out due 
# to scale

diffs = fundfairvals .- fundvals
DataFrame(:Year => Mimi.time_labels(m), :DIFF => diffs) |>
@vlplot(
    :line, 
    x = :Year,
    y = :DIFF,
)

In [ ]:
# cumulative emissions value in 2020 - 2040
idxStart = findfirst(x -> x == 2020, Mimi.time_labels(m))
idxEnd = findfirst(x -> x == 2040, Mimi.time_labels(m))
fundfairvals[idxStart:idxEnd]